In [10]:
import logging

from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, StringType, LongType, 
                                IntegerType, DoubleType, DateType, TimestampType, DecimalType, BooleanType)

In [11]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [12]:
try:
    spark = SparkSession.builder \
                        .appName("ECommerce ETL") \
                        .config("spark.jars", "./../../assets/libs/azure-storage-8.6.6.jar,./../../assets/libs/hadoop-azure-3.4.1.jar") \
                        .getOrCreate()
except Exception as error: 
    logger.error(error)
    spark = SparkSession.builder \
                        .appName("ECommerce ETL") \
                        .config("spark.jars.packages", "com.globalmentor:hadoop-bare-naked-local-fs") \
                        .config("spark.jars", "./../../assets/libs/azure-storage-8.6.6.jar") \
                        .config("spark.hadoop.fs.file.impl", "BareLocalFileSystem") \
                        .master("local") \
                        .getOrCreate()

In [13]:
spark

In [14]:
storage_name = "datastore00"
storage_secret = "KI4UuOBI2SXn4SiGTYVlNGDYI8o/DA/5309yJg77cNG2BGkhb8KZbcePajp78iYjvUXsh5fsxDSG+AStohtiQw=="

spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_name), storage_secret)

In [6]:
# dbutils.fs.mount(source=f"wasbs://raw@{storage_name}.blob.core.windows.net", mount_point="/mnt/raw", extra_configs={f"fs.azure.account.key.{storage_name}.blob.core.windows.net": storage_secret})

In [ ]:
# Create Schema Variables

In [ ]:
categories_schema = """category_id long"""

In [ ]:
products_schema = """product_id long, product_name string, category_id int, brand string, price decimal(10, 2), 
                     cost decimal(10, 2), stock_quantity long, weight_kg decimal(8, 2), dimensions string, 
                     description string, is_active boolean, created_at timestamp
                  """


In [ ]:
customers_schema = StructType([
                                StructField("customer_id", LongType(), False),
                                StructField("email", StringType(), False),
                                StructField("first_name", StringType(), False),
                                StructField("last_name", StringType(), False),
                                StructField("phone", StringType(), True),
                                StructField("date_of_birth", DateType(), False),
                                StructField("gender", StringType(), False),
                                StructField("country", StringType(), False),
                                StructField("city", StringType(), False),
                                StructField("postal_code", StringType(), False),
                                StructField("address", StringType(), False),
                                StructField("registration_date", TimestampType(), False),
                                StructField("last_login", TimestampType(), False),
                                StructField("is_active", BooleanType(), False),
                                StructField("customer_segment", StringType(), False),
                                StructField("marketing_consent", BooleanType(), False)
                            ])


In [ ]:
orders_schema = StructType([
                            StructField("order_id", LongType(), False),
                            StructField("customer_id", IntegerType(), False),
                            StructField("order_date", TimestampType(), False),
                            StructField("status", StringType(), False),
                            StructField("payment_method", StringType(), False),
                            StructField("shipping_address", StringType(), False),
                            StructField("billing_address", StringType(), False),
                            StructField("discount_amount", DecimalType(10, 2), False),
                            StructField("tax_amount", DecimalType(10, 2), False),
                            StructField("shipping_cost", DecimalType(10, 2), False),
                            StructField("total_amount", DecimalType(10, 2), False),
                            StructField("subtotal", DecimalType(10, 2), False),
                            StructField("currency", StringType(), False),
                            StructField("created_at", TimestampType(), False),
                            StructField("updated_at", TimestampType(), False)
                        ])


In [ ]:
order_items_schema = StructType([
                                StructField("order_item_id", LongType(), False),
                                StructField("order_id", IntegerType(), False),
                                StructField("product_id", IntegerType(), False),
                                StructField("quantity", IntegerType(), False),
                                StructField("unit_price", DecimalType(10, 2), False),
                                StructField("line_total", DecimalType(10, 2), False),
                                StructField("discount_amount", DecimalType(10, 2), False)
                            ])


In [ ]:
reviews_schema = StructType([
                            StructField("review_id", LongType(), False),
                            StructField("customer_id", IntegerType(), False),
                            StructField("product_id", IntegerType(), False),
                            StructField("rating", IntegerType(), False),
                            StructField("title", StringType(), False),
                            StructField("comment", StringType(), True),
                            StructField("is_verified_purchase", BooleanType(), False),
                            StructField("helpful_votes", IntegerType(), False),
                            StructField("created_at", TimestampType(), False)
                        ])


In [ ]:
inventory_logs_schema = StructType([
                                    StructField("log_id", LongType(), False),
                                    StructField("product_id", IntegerType(), False),
                                    StructField("movement_type", StringType(), False),
                                    StructField("quantity_change", IntegerType(), False),
                                    StructField("reason", StringType(), False),
                                    StructField("timestamp", TimestampType(), False),
                                    StructField("reference_id", IntegerType(), False),
                                    StructField("notes", StringType(), True)
                                ])

In [15]:
raw_input_path = f"wasbs://raw@{storage_name}.blob.core.windows.net"

# Load a CSV file into a DataFrame

In [ ]:
categories_df = spark.read.csv(raw_input_path + "/ecommerce/categories.csv", \
                               header=True, \
                               inferSchema=True, \
                               timestampFormat="yyyy-MM-dd'T'HH:mm:ss", \
                               dateFormat="yyyy-MM-dd")

Py4JJavaError: An error occurred while calling o52.csv.
: java.lang.NoClassDefFoundError: org/eclipse/jetty/util/ajax/JSON$Convertor
	at org.apache.hadoop.fs.azure.NativeAzureFileSystem.createDefaultStore(NativeAzureFileSystem.java:1485)
	at org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1410)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3615)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:55)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDat

In [ ]:
products_df = spark.read.csv(raw_input_path + "/ecommerce/products.csv", header=True, schema=products_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


In [ ]:
customers_df = spark.read.csv(raw_input_path + "/ecommerce/customers.csv", header=True, schema=customers_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


In [ ]:
orders_df = spark.read.csv(raw_input_path + "/ecommerce/orders*.csv", header=True, schema=orders_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


In [ ]:
order_items_df = spark.read.csv(raw_input_path + "/ecommerce/order_items.csv", header=True, schema=order_items_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


In [ ]:
reviews_df = spark.read.csv(raw_input_path + "/ecommerce/reviews*.csv", header=True, schema=reviews_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


In [ ]:
inventory_logs_df = spark.read.csv(raw_input_path + "/ecommerce/inventory_logs.csv", header=True, schema=inventory_logs_schema, timestampFormat="yyyy-MM-dd'T'HH:mm:ss", dateFormat="yyyy-MM-dd")


# Print Schemas

In [ ]:
categories_df.printSchema()


In [ ]:
products_df.printSchema()


In [ ]:
customers_df.printSchema()


In [ ]:
orders_df.printSchema()


In [ ]:
order_items_df.printSchema()


In [ ]:
reviews_df.printSchema()


In [ ]:
inventory_logs_df.printSchema()

In [0]:
# %ls /mnt/raw/ecommerce/massive

In [17]:
spark.stop()